In [1]:
import os
import pandas as pd
import numpy as np
import requests
import pandas as pd
from scipy.stats import hypergeom
from tqdm import tqdm
import gseapy as gp
currdir=os.getcwd()
parent = os.path.dirname(currdir)
gparent = os.path.dirname(parent)
parent

'/home/poojaparameswaran/Documents/SoderlingLab/SpatialProteomics'

In [ ]:
KOGENE = 'LRRK2'
PATH_OF_INT = 'ParkinsonsDisease'
modules_file = f"{parent}/tables/{KOGENE}_moduleResults_KEGGortholog.csv"
all_modules = pd.read_csv(modules_file)
symbol = all_modules[['Symbol', 'KEGG_Ortholog']]
symbol  

In [3]:

url = 'https://rest.kegg.jp/get/mmu05012'
response = requests.get(url)

# The response will contain the list of genes associated with the mmu05012 pathway
data = response.text

In [4]:
after_GENE = data.split("GENE")
Gene_and_Junk = after_GENE[-1]
GENEINFO = Gene_and_Junk.split("COMPOUND")[0]


In [5]:
lines = GENEINFO.strip().split('\n')

# Parse each line into structured data
data = []
for line in lines:
    parts = line.split('  ')
    parts = [p for p in parts if p]  # Remove empty strings

    if len(parts) >= 2:
        gene_id = parts[0].strip()
        description = parts[1].strip()
        gene_name, description = description.split(";")[0] , description.split(";")[-1]
        desc, KO = description.split("KO:")[0], description.split("KO:")[-1]
        KO = KO.split("]")[0]
        data.append({'Entrez': int(gene_id), 'Symbol': gene_name, "Description": desc,
                     "KEGG_Ortholog": KO})
    else:
        print(parts)
        data.append({'Entrez': int(parts.split(";")[0]), 'Symbol': parts.split(";")[-1]})

PDDF = pd.DataFrame(data)

PDDF.to_csv("ParkinsonsDiseaseGENES.csv")
PDDF

,Entrez,Symbol,Description,KEGG_Ortholog
0,11540,Adora2a,adenosine A2a receptor [,K04266
1,14680,Gnal,"guanine nucleotide binding protein, alpha sti...",K04633
2,14683,Gnas,GNAS complex locus [,K04632
3,13489,Drd2,dopamine receptor D2 [,K04145
4,14677,Gnai1,G protein subunit alpha i1 [,K04630
...,...,...,...,...
261,26419,Mapk8,mitogen-activated protein kinase 8 [,K04440
262,26420,Mapk9,mitogen-activated protein kinase 9 [,K04440
263,26414,Mapk10,mitogen-activated protein kinase 10 [,K04440
264,20655,Sod1,"superoxide dismutase 1, soluble [",K04565


In [6]:
df_1 = all_modules.loc[all_modules['Membership'] == 2]
all_modules['KEGG_Ortholog'].dtype
# PDDF['KEGG ortholog'].dtype
all_modules

,Unnamed: 0,UniProt,Entrez,Symbol,Membership,KEGG_Ortholog
0,0,P43274,50709,H1f4,0,K11275
1,1,P62137,19045,Ppp1ca,0,K06269
2,2,P70670,17938,Naca,0,K03626
3,3,Q6NVF0,320634,Ocrl,0,K01099
4,4,Q6P549,16332,Inppl1,0,K15909
...,...,...,...,...,...,...
1952,1952,P28658,54138,Atxn10,41,K19323
1953,1953,Q9CVB6,76709,Arpc2,41,K05758
1954,1954,Q9WVG6,59035,Carm1,41,K05931
1955,1955,Q9Z1Z0,56041,Uso1,41,FAIL


1. Calculate FoldEnrichment
2. Hypergeometric test to determine significancer of ovelrap. yielding P-val
3. Adjust p value for multiple testin (FDR) since doing multiple memberships
    - benjamini-hochberg procedure to control FDR.

In [13]:
def get_EnrichmentAnalysis(module_num, module_df,all_modules, KEGGpathwaydf, comparison='Symbol'):
    overlap = set(KEGGpathwaydf[comparison]).intersection(set(module_df[comparison]))
    overlap = list(KEGGpathwaydf[KEGGpathwaydf[comparison].isin(overlap)]['Symbol'])
    n_overlap = len(overlap) ##winnersfromA
    n_module = len(module_df[comparison]) ## teamA
    n_parkinsons = len(KEGGpathwaydf[comparison]) #totalwinners
    n_total = len(all_modules[comparison]) # total runners
    fold_enrichment = (n_overlap / n_module) / (n_parkinsons / 30702)
    ## probability overlap num is significant. is it more overlap than ranodom chance?
    ## https://www.youtube.com/watch?v=Go3qssiBzuY
    probability = hypergeom.pmf(n_overlap, 30702, n_parkinsons, n_module)
    # print(probability)
    p_value = 1- hypergeom.cdf(n_overlap, 30702, n_parkinsons, n_module) ## 30702 genes in mouse
    return module_num, n_module, fold_enrichment, probability, p_value, overlap

DIY_enrichment_df = pd.DataFrame(columns=['Module Number', '# Genes in Module',
                                          'Fold Enrichment', 'Probability', 'P-Value', 'Overlap Genes']
                                 )
for member in range(0, max(all_modules['Membership'].unique())+1):
    my_mod = all_modules[all_modules['Membership'] == member]
    mod_num, n_genes_module, fold_enrich, prob, p_val, overlap = get_EnrichmentAnalysis(member, my_mod, all_modules, PDDF, 
                                                                        comparison='Entrez') ## comparison can be entrez,KeggOrtholog
    
    temp_df = pd.DataFrame([{'Module Number': mod_num, '# Genes in Module': n_genes_module,
                             'Fold Enrichment': fold_enrich, 
                             'Probability': prob, 'P-Value': p_val, 'Overlap Genes':overlap}])
    DIY_enrichment_df = pd.concat([DIY_enrichment_df, temp_df], ignore_index=True)

DIY_enrichment_df

/var/folders/pf/xpjm6sg57kgb63302zfdzxyr0000gn/T/ipykernel_2975/1008924470.py:27: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  DIY_enrichment_df = pd.concat([DIY_enrichment_df, temp_df], ignore_index=True)


,Module Number,# Genes in Module,Fold Enrichment,Probability,P-Value,Overlap Genes
0,0,16,0.000000,8.700009e-01,1.299991e-01,[]
1,1,154,2.248462,1.043650e-01,4.539885e-02,"[Rps27a, Psmd3, Psmd4]"
2,2,149,20.140586,2.028027e-26,0.000000e+00,"[Ndufv1, Ndufa1, Ndufa4, Ndufa9, Ndufa10, Nduf..."
3,3,118,1.956289,1.892051e-01,8.329802e-02,"[Uba52, Eif2s1]"
4,4,100,5.771053,1.571024e-03,2.412625e-04,"[Gnal, Gnas, Gnai1, Adcy5, Itpr1]"
5,5,92,2.509153,1.437286e-01,4.599513e-02,"[Psmb1, Ppif]"
6,6,89,0.000000,4.604429e-01,5.395571e-01,[]
7,7,88,6.558014,9.059549e-04,1.195820e-04,"[Uba1, Uchl1, Cox4i1, Vdac1, Park7]"
8,8,85,10.863158,7.173058e-07,5.571037e-08,"[Psmc1, Psmc4, Psmc3, Camk2d, Camk2g, Camk2b, ..."
9,9,81,7.124756,6.291022e-04,7.532017e-05,"[Prkaca, Psmd12, Psmd11, Psmd6, Psmd14]"


In [ ]:
SAVE_DIY_FILE = f"enrichments_DIY/{KOGENE}_{PATH_OF_INT}_hypergeometricDIY.csv"
DIY_enrichment_df.to_csv(SAVE_DIY_FILE)


In [ ]:
def gseapy_analysis(module_num, module_df, comparison='Symbol'):
    gene_list = list(module_df[comparison])
    # Custom Parkinson's disease-associated gene set
    # Perform enrichment analysis
    enr = gp.enrichr(gene_list=list(gene_list),
                            gene_sets=["KEGG_2019_Mouse", 'HDSigDB_Mouse_2021',
                                       'KOMP2_Mouse_Phenotypes_2022', 'Mouse_Gene_Atlas'],
                            outdir= None) #'plots')  # Set to None if you don't want to save results to disk
    enr_df = enr.results
    parkinson_disease_rows = enr_df[enr_df['Term'].str.lower().str.contains("parkinson", na=False)]
    parkinson_disease_rows.insert(0, 'Module_Number', module_num)
    parkinson_disease_rows.insert(1, "# Genes in Module", len(module_df[comparison]))
    return parkinson_disease_rows

GSEA_enrichment_df = pd.DataFrame()
for member in range(0, max(all_modules['Membership'].unique())+1):
    my_mod = all_modules[all_modules['Membership'] == member]
    res = gseapy_analysis(member, my_mod, comparison='Symbol') ## comparison can be entrez,KeggOrtholog
    GSEA_enrichment_df = pd.concat([GSEA_enrichment_df, res], ignore_index=True)

In [20]:
GSEA_enrichment_df

,Module_Number,# Genes in Module,Term,Overlap,P-value,Adjusted P-value,Old P-value,Old Adjusted P-value,Odds Ratio,Combined Score,Genes,Gene_set
0,0,16,Genes Down-Regulated In Parkinson Disease Mark...,2/1054,0.205114,0.481848,0,0,2.570885,4.072765,ARHGAP44;OCRL,HDSigDB_Mouse_2021
1,0,16,Parkinson Disease Markers In Human Substantia ...,3/2327,0.283145,0.514762,0,0,1.753608,2.212695,ARHGAP44;OCRL;RAF1,HDSigDB_Mouse_2021
2,0,16,Genes Up-Regulated In Parkinson Disease Marker...,1/1278,0.652481,0.786837,0,0,0.976612,0.416988,RAF1,HDSigDB_Mouse_2021
3,1,154,Genes Up-Regulated In Parkinson Disease Marker...,9/1278,0.656859,0.914648,0,0,0.908633,0.381885,SRRM2;NXF1;COPA;ERCC3;CXXC1;LUC7L;PHRF1;CYTH1;...,HDSigDB_Mouse_2021
4,1,154,Parkinson Disease Markers In Human Substantia ...,15/2327,0.803606,0.999996,0,0,0.818408,0.178942,SRRM2;COPA;OSBPL6;DGKB;CXXC1;CHD5;DDX10;PHRF1;...,HDSigDB_Mouse_2021
...,...,...,...,...,...,...,...,...,...,...,...,...
108,35,10,Parkinson Disease Markers In Human Substantia ...,1/2327,0.709815,0.778737,0,0,0.843795,0.289212,LLGL1,HDSigDB_Mouse_2021
109,40,5,Genes Down-Regulated In Parkinson Disease Mark...,2/1054,0.024942,0.225226,0,0,12.004436,44.310877,PLCL2;WASF3,HDSigDB_Mouse_2021
110,40,5,Parkinson Disease Markers In Human Substantia ...,2/2327,0.106514,0.357450,0,0,5.066667,11.346711,PLCL2;WASF3,HDSigDB_Mouse_2021
111,41,5,Genes Down-Regulated In Parkinson Disease Mark...,1/1054,0.237173,0.474630,0,0,4.497151,6.471238,ATXN10,HDSigDB_Mouse_2021


In [21]:
SAVE_GSEA_FILE = f"enrichments_GSEA/{KOGENE}_{PATH_OF_INT}_GSEApy.csv"
GSEA_enrichment_df.to_csv(SAVE_GSEA_FILE)
GSEA_enrichment_df

,Module_Number,# Genes in Module,Term,Overlap,P-value,Adjusted P-value,Old P-value,Old Adjusted P-value,Odds Ratio,Combined Score,Genes,Gene_set
0,0,16,Genes Down-Regulated In Parkinson Disease Mark...,2/1054,0.205114,0.481848,0,0,2.570885,4.072765,ARHGAP44;OCRL,HDSigDB_Mouse_2021
1,0,16,Parkinson Disease Markers In Human Substantia ...,3/2327,0.283145,0.514762,0,0,1.753608,2.212695,ARHGAP44;OCRL;RAF1,HDSigDB_Mouse_2021
2,0,16,Genes Up-Regulated In Parkinson Disease Marker...,1/1278,0.652481,0.786837,0,0,0.976612,0.416988,RAF1,HDSigDB_Mouse_2021
3,1,154,Genes Up-Regulated In Parkinson Disease Marker...,9/1278,0.656859,0.914648,0,0,0.908633,0.381885,SRRM2;NXF1;COPA;ERCC3;CXXC1;LUC7L;PHRF1;CYTH1;...,HDSigDB_Mouse_2021
4,1,154,Parkinson Disease Markers In Human Substantia ...,15/2327,0.803606,0.999996,0,0,0.818408,0.178942,SRRM2;COPA;OSBPL6;DGKB;CXXC1;CHD5;DDX10;PHRF1;...,HDSigDB_Mouse_2021
...,...,...,...,...,...,...,...,...,...,...,...,...
108,35,10,Parkinson Disease Markers In Human Substantia ...,1/2327,0.709815,0.778737,0,0,0.843795,0.289212,LLGL1,HDSigDB_Mouse_2021
109,40,5,Genes Down-Regulated In Parkinson Disease Mark...,2/1054,0.024942,0.225226,0,0,12.004436,44.310877,PLCL2;WASF3,HDSigDB_Mouse_2021
110,40,5,Parkinson Disease Markers In Human Substantia ...,2/2327,0.106514,0.357450,0,0,5.066667,11.346711,PLCL2;WASF3,HDSigDB_Mouse_2021
111,41,5,Genes Down-Regulated In Parkinson Disease Mark...,1/1054,0.237173,0.474630,0,0,4.497151,6.471238,ATXN10,HDSigDB_Mouse_2021


In [ ]:
un = res.results['Term'].unique()
parkinson_disease_rows = res.results[res.results['Term'].str.contains("Parkinson disease", na=False)]

# Display the filtered rows
print(parkinson_disease_rows)

In [64]:
gene_set_libraries = gp.get_library_name()
for x in gene_set_libraries:
    if "mouse" in x.lower():
        print(x)
gene_set_mouse = "KEGG_2019_Mouse"

HDSigDB_Mouse_2021
KEGG_2019_Mouse
KOMP2_Mouse_Phenotypes_2022
Mouse_Gene_Atlas
RNAseq_Automatic_GEO_Signatures_Mouse_Down
RNAseq_Automatic_GEO_Signatures_Mouse_Up
WikiPathways_2019_Mouse


Some mouse libraries: HDSigDB_Mouse_2021
KEGG_2019_Mouse
KOMP2_Mouse_Phenotypes_2022
Mouse_Gene_Atlas
RNAseq_Automatic_GEO_Signatures_Mouse_Down
RNAseq_Automatic_GEO_Signatures_Mouse_Up
WikiPathways_2019_Mouse